# OOI Sea Lion TensorFlow Lite Training Example

Author: Ian Black\
Date: 2022-04-28

If you are looking for an example on how to use a sealion model, you can find it [here](https://github.com/oceanobservatories/ooi-data-explorations/tree/master/python/examples/tensorflow).

#### What is the goal of this example?
The goal of this example is to show you how to perform transfer learning on a TensorFlow Lite object detection model. You will be using images from the [OOI CE07SHSM](https://oceanobservatories.org/site/ce07shsm/) buoy to develop a model that can detect a sea lion if a sea-lion is on the buoy. The dataset used to train and validate this model consists of images of sea lions captured on the buoy between 2018 and 2020.  

#### What will the model run on?
This example utilizes the efficientdet_lite0 model. TensorFlow Lite models are optimized to run on mobile and edge devices. However, training them requires a machine that does not have ARM architecture.

#### Why might model creation and validation  fail at higher batch sizes?
This may be a function of your system hardware and the size of the images used in the training data. The efficientdet lite0 model really prefers to have images with a size of 512x512 pixels. The images in the training dataset that come off the OOI buoy cameras have a size of 960x540 pixels and are resized in the model creation process, so running in large batch sizes may result in Out of Memory errors.
***

## Downloading Data

If you don't want to spend hours labeling data, you can download training, validation, and test data from [Kaggle](https://www.kaggle.com/datasets/blackia/ooi-sealions). There you will find folders that contain images and xml files in the Pascal VOC format. There are folders for training the model (658 images), validating the model (89 images), and testing the model (11 images). Simply download the data and extract it to a directory of choice. Once downloaded, make note of the data path in the User Defined Variables section.

## User Defined Variables

You may need to change the following variables to match that of your own operating system.

In [1]:
training_data_location = 'C:/Users/Ian/Desktop/ooi-sealions/train'
validation_data_location = 'C:/Users/Ian/Desktop/ooi-sealions/val'

model_save_location = 'C:/Users/Ian/Desktop/ooi-sealions/models'
model_save_name = 'ooi_sealions'

## Imports

If you don't already have numpy, tensorflow, tflite-model-maker, and tflite-support installed, you can install them with pip.

```
pip install numpy
pip install tensorflow
pip install tflite-model-maker
pip install tflite-support
```

And if you have a GPU you can install the GPU version of tensorflow. Note, that this requires additional [GPU support](https://www.tensorflow.org/install/gpu) setup, which may take some time.

```
pip install tensorflow-gpu
```

In [2]:
import numpy as np
import os

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

from absl import logging
logging.set_verbosity(logging.ERROR)

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

## Loading Data

In [3]:
train_dir = os.path.normpath(training_data_location)
val_dir = os.path.normpath(validation_data_location)
labels = ['sealion']

training_data = object_detector.DataLoader.from_pascal_voc(train_dir,train_dir,labels)
validation_data = object_detector.DataLoader.from_pascal_voc(val_dir,val_dir,labels)

## Identifying the Model

This example uses efficientdet_lite0. Transfer learning is performed using tflite-model-maker, enabling our recreation of the model to detect sealions.

In [4]:
tflite_model = 'efficientdet_lite0'  #You could do 0-4, but values greater than 0 require additional hardware.
spec = model_spec.get(tflite_model)

## Training the Model

In [5]:
batch_size = 8
epochs = 10

model = object_detector.create(training_data, 
                               model_spec=spec, 
                               batch_size=batch_size, 
                               train_whole_model=True, 
                               epochs=epochs, 
                               validation_data=validation_data)

Epoch 1/10
82/82 [==============================] - 40s 251ms/step - det_loss: 1.3438 - cls_loss: 0.7620 - box_loss: 0.0116 - reg_l2_loss: 0.0630 - loss: 1.4068 - learning_rate: 0.0090 - gradient_norm: 2.1388 - val_det_loss: 1.7132 - val_cls_loss: 1.1305 - val_box_loss: 0.0117 - val_reg_l2_loss: 0.0630 - val_loss: 1.7762
Epoch 2/10
82/82 [==============================] - 25s 309ms/step - det_loss: 0.8996 - cls_loss: 0.4464 - box_loss: 0.0091 - reg_l2_loss: 0.0631 - loss: 0.9626 - learning_rate: 0.0093 - gradient_norm: 3.0295 - val_det_loss: 1.1395 - val_cls_loss: 0.6732 - val_box_loss: 0.0093 - val_reg_l2_loss: 0.0631 - val_loss: 1.2027
Epoch 3/10
82/82 [==============================] - 26s 315ms/step - det_loss: 0.7804 - cls_loss: 0.3960 - box_loss: 0.0077 - reg_l2_loss: 0.0632 - loss: 0.8436 - learning_rate: 0.0082 - gradient_norm: 3.3169 - val_det_loss: 1.1855 - val_cls_loss: 0.7017 - val_box_loss: 0.0097 - val_reg_l2_loss: 0.0632 - val_loss: 1.2487
Epoch 4/10
82/82 [=============

## Evaluating the Model

TensorFlow Lite provides a handy function for evaluating a trained model. If we look at the output of the cell below, we can see that the AP (or Average Precision) is around 0.40. Not great, but we only trained for a few epochs.

In [6]:
model.evaluate(validation_data,batch_size = batch_size)

12/12 [==============================] - 5s 258ms/step



{'AP': 0.41080743,
 'AP50': 0.8367011,
 'AP75': 0.3563422,
 'APs': -1.0,
 'APm': 0.21739653,
 'APl': 0.4192642,
 'ARmax1': 0.27732557,
 'ARmax10': 0.5604651,
 'ARmax100': 0.5883721,
 'ARs': -1.0,
 'ARm': 0.41428572,
 'ARl': 0.5957576,
 'AP_/sealion': 0.41080743}

## Saving the Model

In [7]:
export_dir = os.path.normpath(model_save_location)
tflite_filename = '.'.join((model_save_name,'tflite'))

model.export(export_dir=export_dir, tflite_filename=tflite_filename)
model.evaluate_tflite(os.path.join(export_dir,tflite_filename), validation_data)

89/89 [==============================] - 342s 4s/step



{'AP': 0.3876623,
 'AP50': 0.8281781,
 'AP75': 0.27713627,
 'APs': -1.0,
 'APm': 0.21022762,
 'APl': 0.39606765,
 'ARmax1': 0.2761628,
 'ARmax10': 0.51337206,
 'ARmax100': 0.53720933,
 'ARs': -1.0,
 'ARm': 0.31428573,
 'ARl': 0.5466667,
 'AP_/sealion': 0.3876623}

## Check

The next cell will check to see if the model was actually created.

In [8]:
model_filepath = os.path.normpath(os.path.join(model_save_location,model_save_name+'.tflite'))
if os.path.exists(model_filepath):
    print(f"Found model at {model_filepath}.")
    print(f"Congratulations on completing this example!")
else:
    raise OSError(f"Unable to find model at {model_filepath}")

Found model at C:\Users\Ian\Desktop\ooi-sealions\models\ooi_sealions.tflite.
Congratulations on completing this example!
